Lista 3

In [4]:
import numpy as np
from scipy.linalg import solve

In [36]:
def Lu(mat):
    mat = mat.astype(float)
    n = len(mat)
    L, U = np.zeros((n,n)), np.zeros((n,n))

    for i in range(n):
        for k in range(i, n):
            sum_ = 0
            for j in range(i):
                sum_ += (L[i][j] * U[j][k])
            U[i][k] = mat[i][k] - sum_

        for k in range(i, n):
            if (i == k):
                L[i][i] = 1 
            else:
                sum_ = 0
                for j in range(i):
                    sum_ += (L[k][j] * U[j][i])
                L[k][i] = (mat[k][i] - sum_) / U[i][i]
    return L, U


In [37]:
def dokladnosc_maszynowa(x_0):
    x = x_0
    while x + 1 != 1:
        x /= 2
    return x

In [38]:
A = np.array([
    [1, 1/2, 1/3, 1/4, 1/5],
    [1/2, 1/3, 1/4, 1/5, 1/6],
    [1/3, 1/4, 1/5, 1/6, 1/7],
    [1/4, 1/5, 1/6, 1/7, 1/8],
    [1/5, 1/6, 1/7, 1/8, 1/9]
])

b = np.array([5,4,3,2,1])

augmented_matrix = np.column_stack((A, b))

In [39]:
def iteracyjne_poprawianie_rozwiazan(A, b):
    L,U = Lu(A)#rozkład LU
    y = solve(L, b)#rozwiązujemy za pomocą LU
    x =  [1,1,1,1,1]
    x_0 = x
    Ax = np.dot(A, x)
    r = (b.astype(np.float64) - Ax.astype(np.float64))#reszty
    u = np.finfo(np.float64).eps * 10**2
    while np.linalg.norm(r,np.inf) > np.linalg.norm(np.dot(A,x), np.inf) * u:
        delta_x = solve(A, r)
        x += delta_x
        r = b.astype(np.float64) - np.dot(A, x).astype(np.float64)
    return x, x_0

In [40]:
print(iteracyjne_poprawianie_rozwiazan(A, b))

(array([   -95.        ,   2160.00000001, -10710.00000003,  17920.00000004,
        -9450.00000002]), [1, 1, 1, 1, 1])


In [41]:
from scipy.linalg import lu, solve

def iterative_refinement(A, b):
    P, L, U = lu(A)
    y = solve(L, np.dot(P, b))  
    x = solve(U, y)              
    x_0 = x.copy()            

    r = b - np.dot(A, x)
    u = np.finfo(np.float64).eps * 10**2
    
    while np.linalg.norm(r, np.inf) > np.linalg.norm(np.dot(A, x), np.inf) * u:
        delta_x = solve(A, r)
        x += delta_x 
        r = b - np.dot(A, x)
        
    return x, x_0


In [49]:
x_nowe, x_stare = iteracyjne_poprawianie_rozwiazan(A,b)
print(f"Metoda 1: \nRozwiązanie początkowe: {x_stare}\nRozwiązanie końcowe: {x_nowe}")

Metoda 1: 
Rozwiązanie początkowe: [1, 1, 1, 1, 1]
Rozwiązanie końcowe: [   -95.           2160.00000001 -10710.00000003  17920.00000004
  -9450.00000002]


In [50]:
x_nowe2, x_stare2 = iterative_refinement(A,b)
print(f"Metoda 2: \nRozwiązanie początkowe: {x_stare2}\nRozwiązanie końcowe: {x_nowe2}")

Metoda 2: 
Rozwiązanie początkowe: [   -95.           2160.00000002 -10710.00000008  17920.00000012
  -9450.00000006]
Rozwiązanie końcowe: [   -95.           2160.         -10710.00000001  17920.00000002
  -9450.00000001]


Zadanie 2

In [17]:
def generate_matrix_A(n):
    mat = np.zeros((n,n))
    for i in range(n):
        mat[i][i] = 4
        if i - 1 >= 0:
            mat[i][i-1] = -1
        if i + 1 < n:
            mat[i][i+1] = -1
        mat[0][-1] = 1
        mat[-1][0] = 1
    return mat
                     

In [18]:
def generate_matrix_b(n):
    b = np.zeros(n)
    b[-1] = 100
    return b

In [6]:
def ldu(mat):
    n = len(mat)
    l, d, u = mat.copy(), mat.copy(), mat.copy()
    for i in range(n):
        for j in range(n):
            if j > i:
                l[i][j] = 0
                d[i][j] = 0
            if j == i:
                l[i][j] = 0
                u[i][j] = 0
            if i > j:
                u[i][j] = 0
                d[i][j] = 0
    return l, d, u 

In [19]:
A,b = generate_matrix_A(5), generate_matrix_b(5)

In [31]:
def gauss_siedel(A, b, error):
    L, D, U = ldu(A)
    n = len(A)
    N = np.linalg.inv(D + L)
    M = np.dot(-N, U)
    x_0 = np.zeros(n)
    x = np.dot(M, x_0) + np.dot(N, b)
    while np.linalg.norm(x-x_0, np.inf) > error:
        x_0 = x
        x = np.dot(M, x_0) + np.dot(N, b)
    if_converges = max(np.linalg.eig(M)[0]) < 1
    return x, if_converges



In [33]:
error = 10**(-4)

In [34]:
result, convergence = gauss_siedel(A, b, error)
print(f"Rozwiązanie dla błędu {error}: x = {result}")
print(f"Zbieżność rozwiązania: {convergence}")

Rozwiązanie dla błędu 0.0001: x = [-7.57576481 -1.51513633  1.51517289  7.57577322 28.78788451]
Zbieżność rozwiązania: True
